In [1]:
import math
import torch
import gpytorch
from torch.autograd import Variable
from gpytorch.kernels import RBFKernel, GridInterpolationKernel
from gpytorch.means import ConstantMean
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.random_variables import GaussianRandomVariable
from gpytorch.lazy import ToeplitzLazyVariable

class KissGPModel(gpytorch.GPModel):
    def __init__(self):
        likelihood = GaussianLikelihood(log_noise_bounds=(-3, 3))
        super(KissGPModel, self).__init__(likelihood)
        self.mean_module = ConstantMean(constant_bounds=(-1, 1))
        covar_module = RBFKernel(log_lengthscale_bounds=(-100, 100))
        covar_module.log_lengthscale.data = torch.FloatTensor([-2])
        self.grid_covar_module = GridInterpolationKernel(covar_module)
        self.initialize_interpolation_grid(300, grid_bounds=[(0, 1)])

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.grid_covar_module(x)
        return GaussianRandomVariable(mean_x, covar_x)

In [2]:
model = KissGPModel()

n = 1000
x = Variable(torch.rand(n))
x2 = Variable(torch.rand(n))
y = Variable(torch.rand(n))
model.condition(x, y)
toep_var = model.forward(x).covar()
model.condition(x2, y)
toep_var2 = model.forward(x2).covar()

rhs = torch.randn(n)

In [3]:
toep_var_eval = toep_var.evaluate().data
toep_var_2_eval = toep_var2.evaluate().data

actual = (toep_var_eval * toep_var_2_eval).matmul(rhs)

In [4]:
from gpytorch.utils import StochasticLQ

iter_num = 15

z = torch.randn(n, 1)
z = z / torch.norm(z, 2, 0)

print z[:, 0].norm()
def closure_1(rhs):
    return toep_var.matmul(Variable(rhs)).data
Q_1, T_1 = StochasticLQ(max_iter=iter_num).lanczos_batch(closure_1, z)
Q_1 = Q_1[0]
T_1 = T_1[0]

z = torch.randn(n, 1)
z = z / z.norm()
def closure_2(rhs):
    return toep_var2.matmul(Variable(rhs)).data
Q_2, T_2 = StochasticLQ(max_iter=iter_num).lanczos_batch(closure_2, z)
Q_2 = Q_2[0]
T_2 = T_2[0]

print Q_1.size()
print Q_1.t().matmul(Q_1)

res = (Q_1.matmul(T_1).matmul(Q_1.t()).matmul(rhs.diag()).matmul(Q_2).matmul(T_2).matmul(Q_2.t())).diag()

0.999999946867
torch.Size([1000, 1])
torch.Size([1000, 1])
torch.Size([1, 1000, 15])
torch.Size([1, 15])
U.size:
torch.Size([1000, 1])
in _batch_mv
torch.Size([1, 1, 1000])
torch.Size([1, 1000])
torch.Size([1, 1])
in _batch_mv
torch.Size([1, 1000, 1])
torch.Size([1, 1])
torch.Size([1, 1000])
torch.Size([1000, 1000])
torch.Size([1000, 1000])
torch.Size([1000, 1000]) torch.Size([1000, 1000]) torch.Size([1000]) torch.Size([1])


RuntimeError: inconsistent tensor size, expected tensor [1] and src [1000] to have the same number of elements, but got 1 and 1000 elements respectively at /Users/soumith/miniconda2/conda-bld/pytorch_1501999754274/work/torch/lib/TH/generic/THTensorCopy.c:86

In [6]:
print ((res - actual).norm() / actual.norm())

15.5798656578
